# How to use regioinvent

Note that running this entire notebook will take you around 2 to 3 hours.

To be able to use regioinvent, you will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9.1/3.10 cutoff
- to download the trade database stored here: https://doi.org/10.5281/zenodo.11583814

In [2]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r'C:\Users\matth\PycharmProjects\Regiopremise')
import regioinvent

The initialization of the Regioinvent class requires 3 arguments:
- the name of the brightway2 project where you store ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9.1' or '3.10')

In [3]:
ei_version = '3.10.1'
year = '2020'

In [4]:
if ei_version == '3.10.1':
    main_ei_version = '3.10'
else:
    main_ei_version = ei_version

In [7]:
regio = regioinvent.Regioinvent(
    bw_project_name=f'ecoinvent{ei_version}', 
    premise_database_name=f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_{year}+truck_carculator', 
    ecoinvent_database_name=f'ecoinvent-{ei_version}-cutoff',
    ecoinvent_version=main_ei_version,
)

In [ ]:
regio.fix_iam_location_codes()

First step of regioinvent is to spatialize the original ecoinvent database. This entails multiple steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the ecoinvent database, based on the location of the process itself
- Fixing water processes of ecoinvent creating imbalances (e.g., apple production in Chile calls irrigation, RoW -> imbalance)

In [8]:
regio.spatialize_my_ecoinvent()

2025-02-07 13:45:41,810 - Regioinvent - INFO - biosphere3_spatialized_flows already exists in this project.
2025-02-07 13:45:41,810 - Regioinvent - INFO - There is already a spatialized version of ecoinvent in your project. If you want to redo spatialization, please delete it and re-run.


This creates two databases in your brightway2 project:
- "biosphere3_spatialized_flows", which contains all the newly created spatialized elementary flows
- "the-name-of-your-ecoinvent-database regionalized", which is a copy of the original ecoinvent but using spatialized elementary flows and boasting new processes to fix water modeling

Because elementary flows are now spatialize, you will need a specific LCIA method to operate with any flow (spatialized or not). The following function imports such methods. Currently available: "IW v2.1". Soon available: "EF v3.1" and "ReCiPe2016 v1.1 (E)".

In [6]:
regio.import_fully_regionalized_impact_method(lcia_method='IW v2.1')

2025-01-30 09:56:10,713 - Regioinvent - INFO - Importing the fully regionalized version of IMPACT World+ v2.1 for ecoinvent3.10.


If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to regionalize consumption markets of ecoinvent, you can run the next function. There are 3 arguments:
- _trade_database_path_ which is the path where you stored the trade database you downloaded from Zenodo: https://doi.org/10.5281/zenodo.11583814
- _regioinvent_database_name_ which is simply the name that the created database will take
- _cutoff_ which is the cut-off (between 0 and 1) beyond which countries will be aggreated as RoW. For more info on what cutoff does, see section "Selection of countries for regionalization" of the Methodology.md file.

In [9]:
regio.regionalize_ecoinvent_with_trade(
    trade_database_path=r'C:\Users\matth\PycharmProjects\Regiopremise\trade_data.db',
    regioinvent_database_name=f'regioinvent_cutoff_{ei_version}_image_SSP2-Base_{year}+truck_carculator',
    cutoff=0.95,
)

Getting activity data


100%|██████████| 48916/48916 [00:01<00:00, 39775.15it/s]


Adding exchange data to activities


100%|██████████| 1431765/1431765 [02:46<00:00, 8584.30it/s] 


Filling out exchange data


100%|██████████| 48916/48916 [00:46<00:00, 1062.10it/s]
2025-02-07 13:50:03,926 - Regioinvent - INFO - Extracting and formatting trade data...
2025-02-07 13:53:18,049 - Regioinvent - INFO - Regionalizing main inputs of traded products of ecoinvent...
100%|██████████| 1982/1982 [35:09<00:00,  1.06s/it]  
2025-02-07 14:28:27,220 - Regioinvent - INFO - Creating consumption markets...
100%|██████████| 1982/1982 [44:34<00:00,  1.35s/it]  
2025-02-07 15:13:02,643 - Regioinvent - INFO - Performing second order regionalization...
2025-02-07 15:13:17,102 - Regioinvent - INFO - Regionalizing the elementary flows of the regioinvent database...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:10:15


Title: Writing activities to SQLite3 database:
  Started: 02/07/2025 15:14:11
  Finished: 02/07/2025 15:24:27
  Total time elapsed: 00:10:15
  CPU %: 86.10
  Memory %: 38.62


2025-02-07 15:40:34,157 - Regioinvent - INFO - Connecting ecoinvent to regioinvent processes...


This automatically wrote the regioinvent database in your brightway project. So you can now go on brightway2 or AB to perform your LCAs normally with regioinvent.